<img src="../Images/DSC_Logo.png" style="width: 400px;">

# Animations

This notebook explores the creation of animations in Python using matplotlib. It demonstrates how to visualize data dynamically, for example, changes in patterns or distributions over time.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rasterio
from matplotlib.animation import FuncAnimation  

### **Example 1: Sine wave**

We start with animating the sine wave.

In [2]:
# Parameters for the sine wave
x = np.linspace(0, 4 * np.pi, 100)  # x values from 0 to 4π
y = np.sin(x)                       # y values as sine of x

First create the figure that we want to animate. Second, we initialize the object on the figure axes that shall be updated in the animation. In the case of the sine wave it is a line. Next, the animation function that shall update the line object is defined and the anmation created using this function.

In [3]:
# 1. Create the figure

# Setup the figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Setting axis limits
ax.set_xlim((0, 4 * np.pi))
ax.set_ylim((-1.5, 1.5))

# Setting axis labels and title
ax.set_xlabel('x')
ax.set_ylabel('sin(x)')
ax.set_title('Animating a Sine Wave')

# 2. Initialize a line object on the axes (to be updated in animation) with the `plot` function
line, = ax.plot([], [], lw=2, color='red')

# 3. Animation function to update the line object
def animate(i):
    line.set_data(x[:i], y[:i])
    return line,

# 4. Create animation using FuncAnimation
anim = FuncAnimation(fig, animate, frames=len(x), interval=100, blit=True)

# Save animation as a GIF file
anim.save('Figures/sine_wave_animation.gif', writer='pillow', dpi=100)

# Don't show
plt.close(fig)

### **Example 2 [continued from B3]: POLARSTERN cruise PS141 master track**

Let's code a similar animation for the master track of the Polarstern expedition. The track is a time series that we can animate. We resample the data from 10 minutes to dayly means:

In [6]:
# Load dataset
path = '../Datasets/PS141_mastertrack.tab' 
track = pd.read_csv(path, 
                    skiprows=21,
                    sep="\t")

# Convert 'Date/Time' to datetime type
track['Date/Time'] = pd.to_datetime(track['Date/Time'])

# Set 'Date/Time' as the index
track.set_index('Date/Time', inplace=True)

# Resample data to daily frequency
daily_track = track.resample('D').mean()

**Exercise:** Create a line animation of the expedition track. Chek out notebook B3 to create the figure in the first step. Save the animation as a GIF file to the "Figures" folder. Hint: `transform=ccrs.PlateCarree()` must be specified in the line object.

In [7]:
# 1. Create the figure 

# Plot setup
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.SouthPolarStereo()})
ax.coastlines(resolution='50m', color='blue')
ax.gridlines(draw_labels=False, linestyle='--', color='gray', alpha=0.5)

# Set extent
ax.set_extent([-180, 180, -66.8, -22.9], crs=ccrs.PlateCarree())

# Add title
ax.set_title('Polarstern Expedition 141 Track (Daily)', fontsize=15)

# 2. Initialize a line object on the axes (to be updated in animation) with the `plot` function
line, = ax.plot([], [], color='red', transform=ccrs.PlateCarree())

# 3. Animation function to update the line object
def animate(i):
    line.set_data(daily_track['Longitude'][:i], daily_track['Latitude'][:i])
    return line,

# 4. Create animation using FuncAnimation
anim = FuncAnimation(fig, animate, frames=len(daily_track), interval=100, blit=True)

# Save the animation as a GIF
anim.save('Figures/polarstern_track_animation.gif', writer='pillow')

# Don't show
plt.close(fig)

In [9]:
len(daily_track)

69

### **Example 3 [continued from B2]: Chemical concentrations in volcanic tephra**

In [10]:
# Load dataset from two excel sheets
path = '../Datasets/Smith_glass_post_NYT_data.xlsx'
majors = pd.read_excel(path)
traces = pd.read_excel(path, sheet_name=1)

# Define the color mapping for each epoch
color_map = {'one':'red', 'two':'blue', 'three':'purple', 'three-b':'orange'}

Showing the relationship between Zr and Th concentrations while making the time periods (epochs) clear is hard with the simple scatter plot from notebook B2. Instead, we will create an animation that displays the concentrations one epoch at a time. Here, we define the figure inside the `update` function that provides the animation. 

There is almost always room for improvement in the layout of visualisations, for example, the position of the legend could be fixed here. 

In [11]:
# Unique epochs to animate
epochs = list(color_map.keys())

# 1. Create the figure
fig, ax = plt.subplots()

# 2. Animation update function with scatter plot
def update(frame):
    ax.clear()  # Clear the current contents of the axis
    epoch = epochs[frame]
    ax.scatter(traces.loc[traces.Epoch == epoch, 'Zr'], 
               traces.loc[traces.Epoch == epoch, 'Th'], 
               color=color_map[epoch], 
               label=epoch)
    ax.set_title(f"Zr and Th Concentrations - Epoch: {epoch}")
    ax.set_xlabel("Zr [ppm]")
    ax.set_ylabel("Th [ppm]")
    ax.legend(title='Epoch')
    ax.set_xlim(traces['Zr'].min(), traces['Zr'].max())
    ax.set_ylim(traces['Th'].min(), traces['Th'].max())

# 3. Create animation using FuncAnimation
anim = FuncAnimation(fig, update, frames=len(epochs), interval=1000, repeat=True)

# Save the animation
anim.save('Figures/volcanic_compositions_animation.gif', writer='pillow', dpi=100)

# Don't show
plt.close(fig)

### **Example 4 [continued from B3]: Köppen-Geiger maps for 1901–2099**

Let's explore the animation of geospatial raster data using the Köppen-Geiger climate zones. By visualizing the transitions between various climate zones over defined periods, we can effectively demonstrate how climate classifications evolve over time.

Create lists of TIFF files and titles for each period and the custom colormap can be done first:

In [12]:
tif_list = [
    '../Datasets/koppen_geiger/koppen_geiger_1p0_1901_1930.tiff',
    '../Datasets/koppen_geiger/koppen_geiger_1p0_1931_1960.tiff',
    '../Datasets/koppen_geiger/koppen_geiger_1p0_1961_1990.tiff',
    '../Datasets/koppen_geiger/koppen_geiger_1p0_1991_2020.tiff',
    '../Datasets/koppen_geiger/koppen_geiger_1p0_2041_2070_ssp245.tiff',
    '../Datasets/koppen_geiger/koppen_geiger_1p0_2071_2099_ssp245.tiff'
]

titles = [
    '1901-1930',
    '1931-1960',
    '1961-1990',
    '1991-2020',
    '2041-2070 (SSP245)',
    '2071-2099 (SSP245)'
]

file_path = '../Datasets/koppen_geiger/koppen_table.csv'
colors = pd.read_csv(file_path)
koppen_colors = [
    (row['Red']/255, row['Green']/255, row['Blue']/255) for idx, row in colors.iterrows()
]
koppen_cmap = plt.cm.colors.ListedColormap(koppen_colors)

The overall structure and logic for creating the animation remain the same, regardless of whether we are dealing with raster data (like images) or vector data (like points, lines, and polygons). However, the specific plotting functions we use differ based on the type of data. For the raster data here, we use the function `imshow`.

In [13]:
# 1. Create the figure and plot the static elements
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.gridlines(draw_labels=True)

# 2. Animation update function with image plot
def update(frame):
    # Load the current raster data
    with rasterio.open(tif_list[frame]) as src:
        data = src.read(1)  # Read the first band
        transform = src.transform  # Get the transformation information

    # Mask the zeros in the data
    masked_data = np.ma.masked_equal(data, 0)

    # Get extent
    width = data.shape[1]
    height = data.shape[0]
    min_lon, min_lat = transform * (0, height)  # Bottom-left corner
    max_lon, max_lat = transform * (width, 0)  # Top-right corner
    extent = [min_lon, max_lon, min_lat, max_lat]

    # Plot the masked data
    img = ax.imshow(masked_data, origin='upper', extent=extent, cmap=koppen_cmap, 
                    transform=ccrs.PlateCarree(), vmin=1, vmax=len(colors))  # Use the number of classes

    # Set title for the current frame
    ax.set_title(f'Köppen-Geiger Climate Classifications - {titles[frame]}')

# 3. Create animation using FuncAnimation
anim = FuncAnimation(fig, update, frames=len(tif_list), interval=1000, repeat=True)

# Save the animation as a GIF file
anim.save('Figures/koppen_geiger_animation.gif', writer='pillow', dpi=100)

# Show the plot
plt.close(fig)  # Close the plot display

### **Example 5 [continued from B3]: ERA5 climate reanalyis**

The function `imshow` can also be used to animate the netCDF climate data introduced in notebook B3. We first load and prepare the multidimensional dataset. We can also determine the constant extent of the dataset that needs entered in the `imshow` function outside the animation update function.

In [14]:
import xarray as xr

# Load the dataset
ERA5 = xr.open_dataset('../Datasets/ERA5_snippet.nc')

# Convert Kelvin to Celsius
ERA5['t2m'] = ERA5['t2m'] - 273.15

# Resample to weekly means
ERA5 = ERA5.resample(time='1W').mean()

# Determine the extent based on the dataset
min_lon, max_lon = ERA5.longitude.min().values, ERA5.longitude.max().values
min_lat, max_lat = ERA5.latitude.min().values, ERA5.latitude.max().values

**Exercise:** One important step is missing to create the animation. Complete the animation by adding the necessary code and also save this animation.

In [15]:
# 1. Create the figure and plot the static elements
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax.coastlines()
ax.gridlines(draw_labels=True)

# 2. Initialize image object on the axes (to be updated in animation)
t2m_plot = ax.imshow(ERA5['t2m'][0, :, :], origin='upper', 
                     extent=[min_lon, max_lon, min_lat, max_lat], 
                     transform=ccrs.PlateCarree(), cmap='viridis', 
                     animated=True)

# 3. Animation update function with image plot
def animate(i):
    t2m_plot.set_array(ERA5['t2m'][i, :, :])
    ax.set_title(f'2-Meter Temperature [°C] on {str(ERA5.time[i].values)[:10]} over Germany')
    
# 4. Create animation using FuncAnimation
anim = FuncAnimation(fig, animate, frames=len(ERA5['time']), blit=False) # interval=100

# Save as a GIF
anim.save('Figures/ERA5_animation.gif', writer='pillow')

# Don't show
plt.close(fig)